In [1]:
import os
from dotenv import load_dotenv
import time
from datetime import datetime
import schedule
import requests

import pandas as pd
import pandas_ta as ta
import numpy as np

from alpha_vantage.timeseries import TimeSeries
from alpha_vantage.foreignexchange import ForeignExchange
load_dotenv()

True

In [2]:
API_KEYS = os.getenv('ALPHA_KEYS')
cc = ForeignExchange(key=API_KEYS)

LINE_TOKEN = os.getenv('LINE_TOKEN')
LINE_URL = 'https://notify-api.line.me/api/notify'
LINE_HEADERS = {'content-type':'application/x-www-form-urlencoded','Authorization':'Bearer ' + LINE_TOKEN }

ts = TimeSeries(key=API_KEYS, output_format='pandas')

In [3]:
### line alert function
def notifyMsg(*msg, url=LINE_URL, headers=LINE_HEADERS):
    r = requests.post(url, headers=headers, data={'message': msg[0]})
    print(r.text)

In [4]:
### create technical strategy
rsi_o2 = ta.Strategy(
    name = 'RSI 14 Days for opposite trade',
    description = 'RSI for alert',
    ta = [
        {'kind': 'ema', 'length': 200},
        {'kind': 'rsi', 'length': 14}
    ]
)

In [5]:
def get_realtime(pair='EURUSD', tf='15min', back_bars=3, tp=0.001, sl=0.001):
    bars, _ = ts.get_intraday(symbol=pair,interval=tf,  outputsize='full')
    df = pd.DataFrame({
        'timestamp': bars.index,
        'open': bars['1. open'],
        'high': bars['2. high'],
        'low': bars['3. low'],
        'close': bars['4. close'],
        'volumn': bars['5. volume']
    }) # convert alpha vantage data to use with rsi strategy
    
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
    df = df.iloc[::-1] # reverse row
    df.reset_index(drop=True, inplace=True)
    df.ta.strategy(rsi_o2)
    
    # loop count latest n bars that rsi < 30 or rsi > 70
    sell_point = 0
    buy_point = 0
    n_bars = back_bars  # number of latest bars to backword lookup
    for index, row in df.tail(n_bars).iterrows():
        if (row['RSI_14'] < 30 and row['EMA_200'] > row['high']): # rsi < 30 and ema 200 > high price then send sell signal
            sell_point = sell_point + 1
        elif (row['RSI_14'] > 70 and row['EMA_200'] < row['low']): # if rsi > 70 and ema 200 < low price then send buy signal
            buy_point = buy_point + 1
    
        if buy_point >= n_bars:
            notifyMsg(f"Buy {pair}\nprice: {str(row['open'])}\nTP: {str(row['open']+tp)}\nSL: {str(row['open']-sl)}")
        elif sell_point >= n_bars:
            notifyMsg(f"Sell {pair}\nprice: {str(row['open'])}\nTP: {str(row['open']-tp)}\nSL: {str(row['open']+sl)}")
    
# get_realtime()

In [6]:
def get_all_realtime():
    get_realtime(pair='GBPUSD', back_bars=1)          # rr 1:1   10 pip=0.001
    get_realtime(pair='USDJPY', back_bars=1, tp=0.11) # rr 1:1.1 10 pip=0.1
    get_realtime(pair='EURUSD', back_bars=1)          # rr 1:1   10 pip=0.001
    
schedule.every(60 * 15).seconds.do(get_all_realtime) 

Every 900 seconds do get_all_realtime() (last run: [never], next run: 2022-01-07 09:37:05)

In [ ]:
while True:
    schedule.run_pending()
    time.sleep(1)

{"status":200,"message":"ok"}
{"status":200,"message":"ok"}
{"status":200,"message":"ok"}
{"status":200,"message":"ok"}
{"status":200,"message":"ok"}
{"status":200,"message":"ok"}


### === test code below ===

In [3]:
data, _ = cc.get_currency_exchange_rate(from_currency='EUR',to_currency='USD')
ts = TimeSeries(key=API_KEYS, output_format='pandas')

In [4]:
data

{'1. From_Currency Code': 'EUR',
 '2. From_Currency Name': 'Euro',
 '3. To_Currency Code': 'USD',
 '4. To_Currency Name': 'United States Dollar',
 '5. Exchange Rate': '1.12876000',
 '6. Last Refreshed': '2022-01-04 07:13:42',
 '7. Time Zone': 'UTC',
 '8. Bid Price': '1.12876000',
 '9. Ask Price': '1.12876000'}

In [5]:
data_ts,_ = ts.get_intraday(symbol='EURUSD',interval='30min', outputsize='compact')
data_ts

,1. open,2. high,3. low,4. close,5. volume
date,,,,,
2022-01-04 02:30:00,1.1293,1.1295,1.1285,1.1287,0.0
2022-01-04 02:00:00,1.1295,1.1298,1.1290,1.1293,0.0
2022-01-04 01:30:00,1.1305,1.1307,1.1292,1.1294,0.0
2022-01-04 01:00:00,1.1304,1.1305,1.1299,1.1305,0.0
2022-01-04 00:30:00,1.1301,1.1306,1.1299,1.1304,0.0
...,...,...,...,...,...
2021-12-31 02:30:00,1.1316,1.1317,1.1308,1.1313,0.0
2021-12-31 02:00:00,1.1311,1.1316,1.1308,1.1315,0.0
2021-12-31 01:30:00,1.1309,1.1313,1.1306,1.1310,0.0


In [ ]:
all_jobs = schedule.get_jobs()
all_jobs